In [30]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets

In [31]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [32]:
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [35]:
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

In [36]:
train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [37]:
train_images, test_images = train_images/255.0, test_images/255.0

In [38]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [39]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [40]:
# CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의
# 텐서(tensor)를 입력으로 받는다
# MNIST 데이터는 (흑백 이미지이기 때문에) 컬러 채널(channel)이 하나지만
# 컬러 이미지는 (R, G, B) 세 개의 채널을 가진다
# 위의 예는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 정의한 것
# (28, 28, 1) 이 값을 첫 번째 층의 input_shape 매개변수로 전달한다

# 각 필터는 각기 다른 feature들을 추출한다
# 어떤 필터는 고양이의 귀를, 어떤 필터는 고양이의 코를 뽑아낸다
# 서로 필터의 역할이 정해져있다
# 정확히 어떤 필터가 어떠한 역할을 하는지는 알 수 없지만
# 이렇게 여러 개 레이어를 구성하면 어떤 feature들의 조합으로
# 고양이가 되는지, 호랑이가 되는지 구분할 수 있게 된다

# filter 마다 찾아내는 특성이 다르다고 보면 된다
# 갈수록 feature가 고차원으로 더 많은 feature를 뽑아낸다?
# 예를 들면 첫 번째 filter는 모서리, edge 를 찾아내고
# 두 번째 filter는 얼굴의 형태
# 세 번째는 fitler는 사람의 몸을 본다든지
# 단계별로 좀 더 추상화 된 것을 인식한다?
# 우리가 어떤 사람을 인식할 때 그 사람의 shape, 테두리를 보고
# 그 다음에 얼굴을 보고, 몸을 보고, 나중에 합쳐서 본다
# 디테일을 보고 디테일을 합쳐서 하나의 그림이 되는 게 레이어의 전체 단계라고 보면 된다
# 디테일하게 큰 단위, 큰 단위...
# CNN이 어떤 식으로 이미지를 인식하는지 시각화 된 자료를 보면 이해하는 데 도움이 될 것



In [41]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)               

In [42]:
# 모델을 완성하려면 마지막 합성곱 층의 출력 텐서(크기 (3, 3, 64))를 하나 이상의 Dense층에 주입하여 분류를 수행
# Dense 층은 Vector(1D)를 입력으로 받는데 현재 출력은 3D Tensor이다
# 먼저 3D 출력을 1D로 펼친다, 그 다음 하나 이상의 Dense층을 그 위에 추가
# MNIST는 10개의 class가 있으므로 마지막에 Dense 층에 10개의 출력과 softmax 활성화 함수를 사용



In [43]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs = 5, validation_data=(test_images, test_labels))

Epoch 1/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.1507 - accuracy: 0.9535 - val_loss: 0.0544 - val_accuracy: 0.9837
Epoch 2/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0468 - accuracy: 0.9862 - val_loss: 0.0438 - val_accuracy: 0.9860
Epoch 3/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0317 - accuracy: 0.9903 - val_loss: 0.0333 - val_accuracy: 0.9887
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0255 - accuracy: 0.9917 - val_loss: 0.0362 - val_accuracy: 0.9886
Epoch 5/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0202 - accuracy: 0.9937 - val_loss: 0.0291 - val_accuracy: 0.9912


In [44]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose = 2)

313/313 - 1s - loss: 0.0291 - accuracy: 0.9912


In [46]:
print(test_acc)

0.9911999702453613
